# Засоби підготовки та аналізу даних
### ФБ-22 Загородній Ярослав
##### Лаболаторна робота №2

In [2]:
import pandas as pd
import urllib.request
from datetime import datetime
import os
from datetime import datetime, date
import csv

In [4]:
Dir = 'CSV_Files'
Url = ('https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={''}&year1=1981&year2=2024&type=Mean')
Indexes= {1: 24, 2: 25, 3: 5, 4: 6, 5: 27, 6: 23, 7: 26, 8: 7, 9: 11, 10: 13, 11: 14, 12: 15, 13: 16, 14: 17, 15: 18, 16: 19, 17: 21, 18: 22, 19: 8, 20: 9, 21: 10, 22: 1, 23: 3, 24: 2, 25: 4, 26: 20, 27: 12}

if not os.path.exists(Dir):
    os.makedirs(Dir)

In [5]:
for province_id in range(1, 28):
    file_already_exists = False  # Прапорець для перевірки, чи вже є файл з ідентифікатором
    file_name_to_check = 'NOAA_' + str(province_id) + '_'  # Ім'я файлу для перевірки

    # Перевіряємо кожен файл у директорії
    for file_name in os.listdir(Dir):
        if file_name_to_check in file_name:
            file_already_exists = True  # Якщо файл існує, встановлюємо прапорець

    if not file_already_exists:
        url = Url.format(Indexes[province_id])
        wp = urllib.request.urlopen(url)
        text = wp.read()
        now = datetime.now()
        date_and_time_time = now.strftime("%Y%m%d%H%M%S")
        file_name = 'NOAA_' + str(province_id) + '_' + date_and_time_time + '.csv'
        file_path = os.path.join(Dir, file_name)
        with open(file_path, 'wb') as out:
            out.write(text)
        print('File for ' + str(province_id) + ' created: ' + file_name)
    else:
        for file_name in os.listdir(Dir):
            if file_name.startswith(file_name_to_check):
                full_file_path = os.path.join(Dir, file_name)
                with open(full_file_path, 'rb') as file:
                    existing_text = file.read()
                url = Url.format(Indexes[province_id])
                wp = urllib.request.urlopen(url)
                new_text = wp.read()
                if existing_text != new_text:
                    with open(full_file_path, 'wb') as file:
                        file.write(new_text)
                    print('File for ' + str(province_id) + ' updated.')
                else:
                    print('File for ' + str(province_id) + ' already exists and was not downloaded.')


print('Success')

File for 1 created: NOAA_1_20240326181753.csv
File for 2 created: NOAA_2_20240326181754.csv
File for 3 created: NOAA_3_20240326181755.csv
File for 4 created: NOAA_4_20240326181756.csv
File for 5 created: NOAA_5_20240326181757.csv
File for 6 created: NOAA_6_20240326181757.csv
File for 7 created: NOAA_7_20240326181758.csv
File for 8 created: NOAA_8_20240326181759.csv
File for 9 created: NOAA_9_20240326181800.csv
File for 10 created: NOAA_10_20240326181801.csv
File for 11 created: NOAA_11_20240326181802.csv
File for 12 created: NOAA_12_20240326181803.csv
File for 13 created: NOAA_13_20240326181804.csv
File for 14 created: NOAA_14_20240326181805.csv
File for 15 created: NOAA_15_20240326181806.csv
File for 16 created: NOAA_16_20240326181807.csv
File for 17 created: NOAA_17_20240326181808.csv
File for 18 created: NOAA_18_20240326181809.csv
File for 19 created: NOAA_19_20240326181810.csv
File for 20 created: NOAA_20_20240326181811.csv
File for 21 created: NOAA_21_20240326181811.csv
File for 2

In [6]:
def read_files_to_dataframe(directory_path, output_file_path):
    # Визначаємо назви колонок та створюємо порожій DataFrame з ними
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Indexes', 'empty']
    dataframe = pd.DataFrame(columns=headers)

    # Проходимо по всіх CSV-файлах у заданій директорії
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            try:
                # Зчитуємо дані з CSV-файлу, пропускаючи перші два рядки та вказуючи назви колонок
                df = pd.read_csv(file_path, skiprows=2, names=headers[:-1])

                df['Year'] = df['Year'].str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')

                # Визначив індекс регіону з імені файлу та додаємо його до DataFrame
                Indexes = int(filename.split('_')[1])
                df['Indexes'] = Indexes
                # Об'єднуємо DataFrame кожного файлу з загальним DataFrame
                dataframe = pd.concat([dataframe, df], ignore_index=True)
                print(f'Successfully read file: {filename}')
            except pd.errors.ParserError:
                print(f'Error reading {filename}: ParserError')
    dataframe.drop(dataframe.loc[dataframe['VHI']==-1.0].index, inplace=True)
    dataframe = dataframe.drop('empty', axis=1)
    dataframe.to_csv(output_file_path, index=False)
    print(f'DataFrame saved to: {output_file_path}')
    return dataframe
result_dataframe = read_files_to_dataframe('CSV_Files', 'data.csv')
print(result_dataframe)



Successfully read file: NOAA_10_20240326181801.csv
Successfully read file: NOAA_11_20240326181802.csv
Successfully read file: NOAA_12_20240326181803.csv
Successfully read file: NOAA_13_20240326181804.csv
Successfully read file: NOAA_14_20240326181805.csv
Successfully read file: NOAA_15_20240326181806.csv
Successfully read file: NOAA_16_20240326181807.csv
Successfully read file: NOAA_17_20240326181808.csv
Successfully read file: NOAA_18_20240326181809.csv
Successfully read file: NOAA_19_20240326181810.csv
Successfully read file: NOAA_1_20240326181753.csv
Successfully read file: NOAA_20_20240326181811.csv
Successfully read file: NOAA_21_20240326181811.csv
Successfully read file: NOAA_22_20240326181812.csv
Successfully read file: NOAA_23_20240326181814.csv
Successfully read file: NOAA_24_20240326181814.csv
Successfully read file: NOAA_25_20240326181816.csv
Successfully read file: NOAA_26_20240326181817.csv
Successfully read file: NOAA_27_20240326181818.csv
Successfully read file: NOAA_2_2

##### Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [5]:
def procedure_1(dataframe, Indexes, year):
    # Визначає підфрейм за значеннями стовпців Року та Регіону
    region_data = dataframe[(dataframe['Indexes'] == Indexes) & (dataframe['Year'] == year)]
    min_vhi_value = region_data['VHI'].min()
    max_vhi_value = region_data['VHI'].max()
    return min_vhi_value, max_vhi_value

df = pd.read_csv('all.csv')

for i in range(1, 29):
    min_vhi, max_vhi = procedure_1(df, Indexes=i, year=1982)
    print(f"Min VHI для області з індексом {i} у 1982: {min_vhi}, Max VHI для області з індексом {i} у 1982: {max_vhi}")


Min VHI для області з індексом 1 у 1982: 26.95, Max VHI для області з індексом 1 у 1982: 48.77
Min VHI для області з індексом 2 у 1982: 19.92, Max VHI для області з індексом 2 у 1982: 59.42
Min VHI для області з індексом 3 у 1982: 29.13, Max VHI для області з індексом 3 у 1982: 54.44
Min VHI для області з індексом 4 у 1982: 23.55, Max VHI для області з індексом 4 у 1982: 53.88
Min VHI для області з індексом 5 у 1982: 22.44, Max VHI для області з індексом 5 у 1982: 50.09
Min VHI для області з індексом 6 у 1982: 30.43, Max VHI для області з індексом 6 у 1982: 64.88
Min VHI для області з індексом 7 у 1982: 28.77, Max VHI для області з індексом 7 у 1982: 73.75
Min VHI для області з індексом 8 у 1982: 26.98, Max VHI для області з індексом 8 у 1982: 55.58
Min VHI для області з індексом 9 у 1982: 24.94, Max VHI для області з індексом 9 у 1982: 47.0
Min VHI для області з індексом 10 у 1982: 29.46, Max VHI для області з індексом 10 у 1982: 68.51
Min VHI для області з індексом 11 у 1982: 26.65, 

##### Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [6]:
def find_years_with_extreme_drought(df, threshold_percentage, selected_years, selected_provinces):
    years_with_extreme_drought = []

    # Рахуємо кількість областей з екстремальною посухою для кожного року
    for year in df['Year'].unique():
        if year not in selected_years:
            continue
        provinces_with_drought = df[(df['Year'] == year) & (df['VHI'] <= 50)]['Indexes'].unique()
        percentage = len(provinces_with_drought) / len(selected_provinces) * 100
        if percentage >= threshold_percentage:
            years_with_extreme_drought.append(year)

    return years_with_extreme_drought

def procedure_2(df, threshold_percentage, selected_years, selected_provinces):
    for province_n in selected_provinces:
        years_with_extreme_drought = find_years_with_extreme_drought(df[df['Indexes'] == province_n], threshold_percentage, selected_years, selected_provinces)
        if years_with_extreme_drought:
            print(f'For province {province_n} extreme drought is in years: {", ".join(map(str, years_with_extreme_drought))}')

# Задайте відсоток порогу, роки та області
threshold_percentage = 20
selected_years = [1982, 1983, 1984]  # Приклад обраних років
selected_provinces = [3, 7, 12, 18, 20]  # Приклад обраних областей

# Виклик функції procedure_2 з заданими параметрами
procedure_2(df, threshold_percentage, selected_years, selected_provinces)


For province 3 extreme drought is in years: 1982.0, 1983.0, 1984.0
For province 7 extreme drought is in years: 1982.0, 1983.0, 1984.0
For province 12 extreme drought is in years: 1982.0, 1983.0, 1984.0
For province 18 extreme drought is in years: 1982.0, 1983.0, 1984.0
For province 20 extreme drought is in years: 1982.0, 1983.0, 1984.0


In [7]:
def procedure_2(df, province_n):
    df_drought = df[(df["Indexes"] == province_n) & (df.VHI <= 50)]
    list_from_ds = list(df_drought.Year)
    years = set()  # Використовуємо множину для унікальних років
    [years.add(str(item)) for item in list_from_ds]  # Додаємо рік до множини
    sorted_years = sorted(years, key=lambda x: float(x))  # Сортуємо роки за зростанням
    print('For the province ' + str(province_n) + ' extreme drought is in:')
    print('Years: ' + ' '.join(sorted_years))

procedure_2(df, 3)


For the province 3 extreme drought is in:
Years: 1982.0 1983.0 1984.0 1985.0 1986.0 1987.0 1988.0 1989.0 1990.0 1991.0 1992.0 1993.0 1994.0 1995.0 1996.0 1997.0 1998.0 1999.0 2000.0 2001.0 2002.0 2003.0 2004.0 2005.0 2006.0 2007.0 2008.0 2009.0 2010.0 2011.0 2012.0 2013.0 2014.0 2015.0 2016.0 2017.0 2018.0 2019.0 2020.0 2021.0 2022.0 2023.0 2024.0


##### Аналогічно для помірних посух

In [8]:
def find_years_with_moderate_drought(df, threshold_percentage, selected_years, selected_provinces):
    years_with_moderate_drought = []

    # Рахуємо кількість областей з помірною посухою для кожного року
    for year in df['Year'].unique():
        if year not in selected_years:
            continue
        provinces_with_drought = df[(df['Year'] == year) & (df['VHI'] <= 35) & (df['VHI'] > 15)]['Indexes'].unique()
        percentage = len(provinces_with_drought) / len(selected_provinces) * 100
        if percentage >= threshold_percentage:
            years_with_moderate_drought.append(year)

    return years_with_moderate_drought

def procedure_moderate_drought(df, threshold_percentage, selected_years, selected_provinces):
    for province_n in selected_provinces:
        years_with_moderate_drought = find_years_with_moderate_drought(df[df['Indexes'] == province_n], threshold_percentage, selected_years, selected_provinces)
        if years_with_moderate_drought:
            print(f'For province {province_n} moderate drought is in years: {", ".join(map(str, years_with_moderate_drought))}')

# Задайте відсоток порогу, роки та області
threshold_percentage = 20
selected_years = [1982, 1983, 2002]  # Приклад обраних років
selected_provinces = [3, 7, 12, 18, 20]  # Приклад обраних областей

# Виклик функції procedure_moderate_drought з заданими параметрами
procedure_moderate_drought(df, threshold_percentage, selected_years, selected_provinces)


For province 3 moderate drought is in years: 1982.0, 1983.0
For province 7 moderate drought is in years: 1982.0, 1983.0, 2002.0
For province 12 moderate drought is in years: 1982.0, 1983.0, 2002.0
For province 18 moderate drought is in years: 1982.0, 1983.0
For province 20 moderate drought is in years: 1982.0, 1983.0, 2002.0


In [9]:
def procedure_3(df, province_n):
    df_drought = df[(df["Indexes"] == province_n) & (df.VHI <= 35) & (df.VHI > 15)]
    list_from_ds = list(df_drought.Year)
    years = []
    [years.append(str(item)) for item in list_from_ds if str(item) not in years]  # Перетворення на рядок
    print('For the province ' + str(province_n) + ' moderate drought is in:')
    print('Years: ' + ' '.join(years))

procedure_3(df, 3)

For the province 3 moderate drought is in:
Years: 1982.0 1983.0 1984.0 1985.0 1986.0 1987.0 1988.0 1989.0 1990.0 1991.0 1992.0 1993.0 1994.0 1995.0 1996.0 1997.0 1998.0 1999.0 2000.0 2001.0 2013.0 2014.0 2018.0 2019.0 2023.0
